Introduction to PyTorch

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

PyTorch provides two high-level features:

1) Tensor computation like numpy with strong GPU acceleration.
2) Deep Neural Networks built on a tape-based autodiff system

In [3]:
x=torch.ones(3,2)
print(x)

y=torch.zeros(3,2)
print(y)

z=torch.rand(3,2)
print(z)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[0.9961, 0.1715],
        [0.4209, 0.4257],
        [0.9781, 0.9163]])


In [4]:
x=torch.empty(3,2)
print(x)

tensor([[1.8971e-35, 0.0000e+00],
        [3.3631e-44, 0.0000e+00],
        [       nan, 0.0000e+00]])


In [5]:
y=torch.zeros_like(x)

In [6]:
y

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

Accessing tensors

In [7]:
x=torch.tensor([[1,2],
                [3,5],
                [7,8]])

# To access the 2nd row and 2nd column element
t=x[1,1]
print(t)

tensor(5)


In [8]:
t.item()   #to get the value

5

In [9]:
y=x.view(2,3)   # Like the reshape operation in numpy
print(y)

tensor([[1, 2, 3],
        [5, 7, 8]])


In [10]:
z=x.view(6,-1)
print(z)

tensor([[1],
        [2],
        [3],
        [5],
        [7],
        [8]])


In [11]:
# To get the size of the tensor we can do:

z.size()

torch.Size([6, 1])



```

NUMPY -> PYTORCH 
PYTORCH -> NUMPY
```



In [12]:
x_np=x.numpy()
print(type(x_np))
print(type(x))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [13]:
a=np.random.randn(5)
a_pt=torch.from_numpy(a)
print(type(a))
print(type(a_pt))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [14]:
%%time
for i in range(100):
  a=np.random.randn(100,100)
  b=np.random.randn(100,100)
  # c=a*b
  c=np.matmul(a,b)


CPU times: user 140 ms, sys: 116 ms, total: 256 ms
Wall time: 144 ms


In [15]:
%%time
for i in range(100):
  a=torch.randn(100,100)
  b=torch.randn(100,100)
  # c=a*b
  c=torch.matmul(a,b)

CPU times: user 27 ms, sys: 0 ns, total: 27 ms
Wall time: 32 ms


### CUDA SUPPORT

In [16]:
print(torch.cuda.device_count())

1


In [17]:
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

Tesla T4


In [18]:
cuda0=torch.device('cuda:0')

In [19]:
%%time
for i in range(1000):
  a=torch.randn(100,100)
  b=torch.randn(100,100)
  # c=a*b
  c=torch.matmul(a,b)

CPU times: user 273 ms, sys: 0 ns, total: 273 ms
Wall time: 274 ms


In [20]:
%%time
for i in range(1000):
  a=torch.randn([100,100],device=cuda0)
  b=torch.randn([100,100],device=cuda0)
  # c=a*b
  c=torch.matmul(a,b)

CPU times: user 3.03 s, sys: 1.31 s, total: 4.34 s
Wall time: 4.35 s


AutoGrad

In [33]:
x=torch.ones([3,2], requires_grad=True)

In [34]:
x

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)

In [35]:
y=x+5
z=y*y+1

In [36]:
sumi=torch.sum(z)
sumi

tensor(222., grad_fn=<SumBackward0>)

In [37]:
# Doing a Backward Pass
sumi.backward()

In [38]:
print(x.grad)

tensor([[12., 12.],
        [12., 12.],
        [12., 12.]])


In [39]:
x=torch.ones([3,2],requires_grad=True)
y=x+5
r=1/(1+torch.exp(-y)) #Sigmoid function
print(r)
s=torch.sum(r)
s.backward()
print(x.grad)

tensor([[0.9975, 0.9975],
        [0.9975, 0.9975],
        [0.9975, 0.9975]], grad_fn=<MulBackward0>)
tensor([[0.0025, 0.0025],
        [0.0025, 0.0025],
        [0.0025, 0.0025]])


In [40]:
x=torch.ones([3,2],requires_grad=True)
y=x+5
r=1/(1+torch.exp(-y)) #Sigmoid function
print(r)
# s=torch.sum(r)
a=torch.ones(3,2)
r.backward(a)
print(x.grad)

tensor([[0.9975, 0.9975],
        [0.9975, 0.9975],
        [0.9975, 0.9975]], grad_fn=<MulBackward0>)
tensor([[0.0025, 0.0025],
        [0.0025, 0.0025],
        [0.0025, 0.0025]])


In [41]:
x=torch.randn([20,1], requires_grad=True)
y=3*x-2


In [42]:
w=torch.tensor([1.], requires_grad=True)
b=torch.tensor([1.], requires_grad=True)

In [43]:
y_hat=w*x+b

In [44]:
loss=torch.sum((y_hat-y)**2)

In [45]:
print(loss)

tensor(292.5909, grad_fn=<SumBackward0>)


In [46]:
loss.backward()

In [47]:
print(w.grad,b.grad) # derivative of loss with respect to w and with respect to b

tensor([-96.8343]) tensor([130.5045])


### Doing it in a loop

In [54]:
learning_rate=0.01
w=torch.tensor([1.], requires_grad=True)
b=torch.tensor([1.], requires_grad=True)


In [55]:
print(w.item(),b.item())
print('********************')
for i in range(10):
  x=torch.randn([20, 1])
  y=3*x-2

  y_hat=w*x+b
  loss=torch.sum((y_hat-y)**2)
  loss.backward()

  with torch.no_grad():
    w-=learning_rate*w.grad
    b-=learning_rate*b.grad

    w.grad.zero_()
    b.grad.zero_()
  
  print(w.item(),b.item())



1.0 1.0
********************
1.4751468896865845 0.331703782081604
1.5946309566497803 -0.35801422595977783
2.0782182216644287 -1.0194096565246582
2.2912938594818115 -1.4155011177062988
2.5187995433807373 -1.654730200767517
2.657292604446411 -1.8051912784576416
2.763023614883423 -1.8711755275726318
2.8767402172088623 -1.9252744913101196
2.9173130989074707 -1.9439148902893066
2.931438684463501 -1.960207462310791


## Every tensor that you create have an argument device=cuda0